In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
data_path = "./ex_all.csv"
custom_palette = {
    "ER": "#1f77b4",      # Blue color
    "BER": "#ff7f0e",     # Orange color
    "PER": "#2ca02c",     # Green color
    "PBER": "#d62728"     # Red color
}

In [3]:
data_df = pd.read_csv(data_path)
data_df = data_df.copy()
data_df_filtered = data_df[data_df["num_env_steps_sampled"] <= 400000]
data_df_filtered["observe"] = data_df_filtered["all"] - data_df_filtered[["store", "sample", "train", "update"]].sum(axis=1)

# 2. Calculate the mean sample_rate for each environment and buffer type
data_df_filtered["num_env_steps_sampled"] = data_df_filtered["num_env_steps_sampled"] * 4
data_df_filtered["sample_rate"] = data_df_filtered["num_env_steps_sampled"] / data_df_filtered["time_total_s"]
mean_sample_rates = data_df_filtered.groupby(['env', 'buffer'])["sample_rate"].mean().reset_index()

# 3. Calculate the percentage difference between ER and BER
per_sample_rates = mean_sample_rates[mean_sample_rates['buffer'] == 'PER']["sample_rate"].values
pber_sample_rates = mean_sample_rates[mean_sample_rates['buffer'] == 'PBER']["sample_rate"].values
percentage_differences = ((pber_sample_rates - per_sample_rates) / per_sample_rates) * 100

# 4. Calculate the average percentage difference across all environments
average_percentage_difference = percentage_differences.mean()

min_max_sampled_per_pber = data_df_filtered[data_df_filtered['buffer'].isin(['PER', 'PBER'])].groupby(['env', 'buffer'])['num_env_steps_sampled'].max().unstack().min(axis=1)
min_max_sampled_ber_er = data_df_filtered[data_df_filtered['buffer'].isin(['BER', 'ER'])].groupby(['env', 'buffer'])['num_env_steps_sampled'].max().unstack().min(axis=1)

data_df_filtered = data_df_filtered.merge(min_max_sampled_per_pber.reset_index(), on='env', how='left')
data_df_filtered.rename(columns={0: 'num_env_steps_sampled_min_max_for_per_pber'}, inplace=True)

data_df_filtered = data_df_filtered.merge(min_max_sampled_ber_er.reset_index(), on='env', how='left')
data_df_filtered.rename(columns={0: 'num_env_steps_sampled_min_max_for_ber_er'}, inplace=True)

filtered_data_per_pber = data_df_filtered[(data_df_filtered['num_env_steps_sampled'] <= data_df_filtered['num_env_steps_sampled_min_max_for_per_pber']) & data_df_filtered['buffer'].isin(['PBER', 'PER'])]
# 5. Calculate the total operation time for each buffer type
columns_to_compare = ["store", "sample", "update"]

per_total_time = filtered_data_per_pber[filtered_data_per_pber['buffer'] == 'PER'][columns_to_compare].sum().sum()
pber_total_time = filtered_data_per_pber[filtered_data_per_pber['buffer'] == 'PBER'][columns_to_compare].sum().sum()

# 6. Calculate the time difference between PER and PBER
time_saved = per_total_time - pber_total_time
percentage_time_saved = (time_saved / per_total_time) * 100
print(average_percentage_difference, percentage_time_saved)

19.6828636610867 72.67446387092832


/tmp/ipykernel_2163/1944421607.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df_filtered["observe"] = data_df_filtered["all"] - data_df_filtered[["store", "sample", "train", "update"]].sum(axis=1)
/tmp/ipykernel_2163/1944421607.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df_filtered["num_env_steps_sampled"] = data_df_filtered["num_env_steps_sampled"] * 4
/tmp/ipykernel_2163/1944421607.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [4]:
min_max_sampled_per_pber

env
BeamRider        1600000
Breakout         1600000
Qbert            1600000
SpaceInvaders    1600000
dtype: int64

In [5]:
data_df = pd.read_csv(data_path)
n = 10
data_df["time(h)"] = data_df["time_total_s"] / 3600
window_size_hours = 0.25
# 2. Define helper functions

def apply_offsets_to_data(data):
    recommended_offsets = {
        'Boxing': 20,
        'Pong': 21,
        'FishingDerby': 100,
        'Frostbite': 5,
        'Enduro': 10,
        'Q*bert': 5
    }
    for env, min_reward in data.groupby('env')['episode_reward_mean'].min().items():
        if env not in recommended_offsets and min_reward < 0:
            recommended_offsets[env] = abs(min_reward) + 1
    data['episode_reward_mean_offset'] = data.apply(lambda row: row['episode_reward_mean'] + recommended_offsets.get(row['env'], 0), axis=1)
    return data

def compute_interval_mean_with_env(env_data):
    env_name = env_data['env'].iloc[0]
    bins = list(np.arange(0, env_data["time(h)"].max() + window_size_hours, window_size_hours))
    env_data["time_interval"] = pd.cut(env_data["time(h)"], bins, labels=bins[:-1], right=False)
    numeric_cols = env_data.select_dtypes(include=[np.number]).columns.tolist()
    interval_data = env_data.groupby('time_interval')[numeric_cols].mean().reset_index()
    interval_data['env'] = env_name
    return interval_data

def compute_avg_reward_last_episodes(env_name, data1, data2, n):
    last_rewards1 = data1[data1['env'] == env_name]['episode_reward_mean'].tail(n).values
    last_rewards2 = data2[data2['env'] == env_name]['episode_reward_mean'].tail(n).values
    avg_reward1 = last_rewards1.mean()
    avg_reward2 = last_rewards2.mean()
    return avg_reward1, avg_reward2

def compute_T_save_for_env(env_name, data1, data2):
    rewards1 = data1[data1['env'] == env_name]['episode_reward_mean'].values
    rewards2 = data2[data2['env'] == env_name]['episode_reward_mean'].values
    if len(rewards1) == 0 or len(rewards2) == 0:
        return None
    times1 = data1[data1['env'] == env_name]['time(h)'].values
    times2 = data2[data2['env'] == env_name]['time(h)'].values
    time_diff_ratios = []
    max_reward = max(rewards1.max(), rewards2.max())
    reward_samples = np.linspace(0, max_reward, 51)
    for target_reward in reward_samples:
        time1 = next((times1[i] for i, r in enumerate(rewards1) if r >= target_reward), None)
        time2 = next((times2[i] for i, r in enumerate(rewards2) if r >= target_reward), None)
        if time1 is not None and time2 is not None:
            time_diff_ratio = (time1 - time2) / time1
            time_diff_ratios.append(time_diff_ratio)
    return np.median(time_diff_ratios) if time_diff_ratios else None

# 3. Main data processing and analysis steps

# Sort data by time
data_df_sorted = data_df.sort_values(by=["env", "time(h)"])

# Split sorted data for ER vs. BER and PER vs. PBER
er_data_sorted = data_df_sorted[data_df_sorted['buffer'] == 'ER']
ber_data_sorted = data_df_sorted[data_df_sorted['buffer'] == 'BER']
per_data_sorted = data_df_sorted[data_df_sorted['buffer'] == 'PER']
pber_data_sorted = data_df_sorted[data_df_sorted['buffer'] == 'PBER']

# Apply offsets to the data
data_df_sorted = apply_offsets_to_data(data_df_sorted)

# Compute interval mean for each buffer type and environment using sorted data
er_interval_data_sorted = pd.concat([compute_interval_mean_with_env(er_data_sorted[er_data_sorted['env'] == env]) for env in er_data_sorted['env'].unique()])
ber_interval_data_sorted = pd.concat([compute_interval_mean_with_env(ber_data_sorted[ber_data_sorted['env'] == env]) for env in ber_data_sorted['env'].unique()])
per_interval_data_sorted = pd.concat([compute_interval_mean_with_env(per_data_sorted[per_data_sorted['env'] == env]) for env in per_data_sorted['env'].unique()])
pber_interval_data_sorted = pd.concat([compute_interval_mean_with_env(pber_data_sorted[pber_data_sorted['env'] == env]) for env in pber_data_sorted['env'].unique()])

# Compute metrics for ER vs. BER and PER vs. PBER using sorted data
results_sorted = []
for env in data_df_sorted['env'].unique():
    R_improve_er_ber = (ber_interval_data_sorted[ber_interval_data_sorted['env'] == env]["episode_reward_mean"].median() - er_interval_data_sorted[er_interval_data_sorted['env'] == env]["episode_reward_mean"].median()) / ber_interval_data_sorted[ber_interval_data_sorted['env'] == env]["episode_reward_mean"].median()
    R_improve_per_pber = (pber_interval_data_sorted[pber_interval_data_sorted['env'] == env]["episode_reward_mean"].median() - per_interval_data_sorted[per_interval_data_sorted['env'] == env]["episode_reward_mean"].median()) / pber_interval_data_sorted[pber_interval_data_sorted['env'] == env]["episode_reward_mean"].median()

    
    T_save_er_ber = compute_T_save_for_env(env, er_data_sorted, ber_data_sorted)
    T_save_per_pber = compute_T_save_for_env(env, per_data_sorted, pber_data_sorted)
    
    avg_er_reward, avg_ber_reward = compute_avg_reward_last_episodes(env, er_data_sorted, ber_data_sorted, n)
    avg_per_reward, avg_pber_reward = compute_avg_reward_last_episodes(env, per_data_sorted, pber_data_sorted, n)
    
    improvement_percentage_er_ber = ((avg_ber_reward - avg_er_reward) / avg_er_reward)  if avg_er_reward != 0 else np.nan
    improvement_percentage_per_pber = ((avg_pber_reward - avg_per_reward) / avg_per_reward)  if avg_per_reward != 0 else np.nan
    
    results_sorted.append({
        'Environment': env,
        'R_improve_ER_BER': R_improve_er_ber,
        'R_improve_PER_PBER': R_improve_per_pber,
        'T_save_ER_BER': T_save_er_ber,
        'T_save_PER_PBER': T_save_per_pber,
        'Improvement_ER_BER': improvement_percentage_er_ber,
        'Improvement_PER_PBER': improvement_percentage_per_pber
    })

# Convert results to a DataFrame and compute mean values
result_df_sorted = pd.DataFrame(results_sorted).set_index('Environment')
mean_values_sorted = result_df_sorted.mean(axis=0, skipna=True)
mean_values_sorted.name = "Mean"
result_df_sorted = pd.concat([result_df_sorted, pd.DataFrame(mean_values_sorted).T])

result_df_sorted

/tmp/ipykernel_2163/3549563666.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  env_data["time_interval"] = pd.cut(env_data["time(h)"], bins, labels=bins[:-1], right=False)
/tmp/ipykernel_2163/3549563666.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  env_data["time_interval"] = pd.cut(env_data["time(h)"], bins, labels=bins[:-1], right=False)
/tmp/ipykernel_2163/3549563666.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

,R_improve_ER_BER,R_improve_PER_PBER,T_save_ER_BER,T_save_PER_PBER,Improvement_ER_BER,Improvement_PER_PBER
BeamRider,0.202360,0.551634,-0.789371,0.513878,0.285287,0.476032
Breakout,-0.185787,0.223589,-0.329945,0.433744,0.048994,-0.178539
Qbert,0.049096,0.301625,0.307731,0.294004,0.041337,0.080461
SpaceInvaders,-0.382286,0.109305,0.173007,0.181701,-0.127374,0.173793
Mean,-0.079154,0.296538,-0.159644,0.355832,0.062061,0.137937


In [6]:
def generate_refined_combined_latex_table(df1, df2):
    header = "\\begin{tabular}{" + "l" + "c"*df1.shape[1] + "c"*df2.shape[1] + "}\n\\toprule"
    
    # Headers with formatted names
    column_names_1 = ["$" + col.split("_")[0] + "_{" + col.split("_")[1] + "}$" for col in df1.columns]
    column_names_2 = ["$" + col.split("_")[0] + "_{" + col.split("_")[1] + "}$" for col in df2.columns]
    combined_columns = column_names_1 + column_names_2
    
    rows = [" & \\multicolumn{3}{c}{ER vs BER} & \\multicolumn{3}{c}{PER vs PBER} \\\\ \\cmidrule(lr){2-4} \\cmidrule(lr){5-7}"]
    rows.append(" & ".join([""] + combined_columns) + " \\\ \midrule")
    
    for index, (row1, row2) in enumerate(zip(df1.iterrows(), df2.iterrows())):
        combined_row = list(row1[1].values) + list(row2[1].values)
        rows.append("{} & {}".format(row1[0], " & ".join(["{:.4f}".format(val) for val in combined_row])) + " \\\ ")
    
    footer = "\\bottomrule\n\\end{tabular}"
    return header + "\n" + "\n".join(rows) + "\n" + footer
# Splitting the dataframe into two: ER vs BER and PER vs PBER

columns_er_ber = ["R_improve_ER_BER", "T_save_ER_BER", "Improvement_ER_BER"]
columns_per_pber = ["R_improve_PER_PBER", "T_save_PER_PBER", "Improvement_PER_PBER"]

df_er_ber = result_df_sorted[columns_er_ber]
df_per_pber = result_df_sorted[columns_per_pber]
combined_latex_table = generate_refined_combined_latex_table(df_er_ber, df_per_pber)
print(combined_latex_table)

\begin{tabular}{lcccccc}
\toprule
 & \multicolumn{3}{c}{ER vs BER} & \multicolumn{3}{c}{PER vs PBER} \\ \cmidrule(lr){2-4} \cmidrule(lr){5-7}
 & $R_{improve}$ & $T_{save}$ & $Improvement_{ER}$ & $R_{improve}$ & $T_{save}$ & $Improvement_{PER}$ \\ \midrule
BeamRider & 0.2024 & -0.7894 & 0.2853 & 0.5516 & 0.5139 & 0.4760 \\ 
Breakout & -0.1858 & -0.3299 & 0.0490 & 0.2236 & 0.4337 & -0.1785 \\ 
Qbert & 0.0491 & 0.3077 & 0.0413 & 0.3016 & 0.2940 & 0.0805 \\ 
SpaceInvaders & -0.3823 & 0.1730 & -0.1274 & 0.1093 & 0.1817 & 0.1738 \\ 
Mean & -0.0792 & -0.1596 & 0.0621 & 0.2965 & 0.3558 & 0.1379 \\ 
\bottomrule
\end{tabular}
